# Basic web scraping and analysis
This script will extract the table information from a wikipedia link and house the information in a data frame for later processing.

In [80]:
import numpy as np
import pandas as pd
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

In [102]:
url = r'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
    
df = pd.DataFrame(columns = ['Postal Code', 'Borough', 'Neighbourhood'])

### Data extraction to data frame

In [103]:
res = requests.get(url).text
soup = BeautifulSoup(res,'lxml')
nulCase = 'Not assigned'
skipRow = False
for items in soup.find('table', class_='wikitable').find_all('tr')[1::1]:
    data = items.find_all(['th','td'])
    postCode = data[0].text[0:-1]
    borough = data[1].text[0:-1]
    neighbourhood = data[2].text[0:-1]

    if borough == nulCase:
        skipRow = True

    if neighbourhood == nulCase and borough != nulCase:
        neighborhood = borough

    newRow = {'Postal Code' : postCode, 'Borough' : borough, 'Neighbourhood' : neighbourhood}
    
    if skipRow == False:
        df = df.append(newRow, ignore_index = True)
    skipRow = False

In [106]:
df.head(15)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Borough



True